## **Homework 3**


2022320009 이수현

이미지는 픽사베이의 무료 이미지를 사용하였습니다


**Instructions**
* This homework focuses on understanding and applying DETR for object detection and attention visualization. It consists of **three questions** designed to assess both theoretical understanding and practical application.

* Please organize your answers and results for the questions below and submit this jupyter notebook as **a .pdf file**.

* **Deadline: 11/14 (Thur) 23:59**

**Reference**
* End-to-End Object Detection with Transformers (DETR): https://github.com/facebookresearch/detr

### **Q1.  Understanding DETR model**

* Fill-in-the-blank exercise to test your understanding of critical parts of the DETR model workflow.



In [ ]:
from torch import nn
class DETR(nn.Module):
    def __init__(self, num_classes, hidden_dim=256, nheads=8,
                 num_encoder_layers=6, num_decoder_layers=6, num_queries=100):
        super().__init__()

        # create ResNet-50 backbone
        self.backbone = resnet50()
        del self.backbone.fc

        # create conversion layer
        self.conv = nn.Conv2d(2048, hidden_dim, 1)

        # create a default PyTorch transformer
        self.transformer = nn.Transformer(
            hidden_dim, nheads, num_encoder_layers, num_decoder_layers)

        # prediction heads, one extra class for predicting non-empty slots
        # note that in baseline DETR linear_bbox layer is 3-layer MLP
        self.linear_class = nn.Linear(hidden_dim, num_classes + 1) #모델이 예측해야하는 총 class수. objet가 없는 경우를 위해 +1
        self.linear_bbox = nn.Linear(hidden_dim, 4)

        # output positional encodings (object queries)
        self.query_pos = nn.Parameter(torch.rand(num_queries, hidden_dim)) #

        # spatial positional encodings
        # note that in baseline DETR we use sine positional encodings
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))

    def forward(self, inputs):
        # propagate inputs through ResNet-50 up to avg-pool layer
        x = self.backbone.conv1(inputs)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        # convert from 2048 to 256 feature planes for the transformer
        h = self.conv(x)

        # construct positional encodings
        H, W = h.shape[-2:]
        pos = torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
        ], dim=-1).flatten(0, 1).unsqueeze(1)

        # propagate through the transformer
        h = self.transformer(pos + 0.1 * h.flatten(2).permute(2, 0, 1),
                             self.query_pos.unsqueeze(1)).transpose(0, 1)



        # finally project transformer outputs to class labels and bounding boxes
        pred_logits = _______
        pred_boxes = _______

        return {'pred_logits': pred_logits,
                'pred_boxes': pred_boxes}

### **takeout messages**

[DETR]

* anchor box 없애고 단순한 구조
    -> 모델이 더 가벼워지고 빠름
* end to end 구조
    * 기존에는 anchor box -> 후처리(NMS)가 필요
    * end to end 구조로 더 단순해짐
* transformer 기반 구조
    * self attention과 cross attention(encoder-decoder attention)을 이용하여 전반적인 관계와 각 객체간의 연관성 학습
* binary matching에 헝가리안 알고리즘을 사용

* __init__ 함수
    * 주요 구조 : 백본 / transformer encoder-decoder / 예측헤드
    * ResNet-50 backbone으로 feature extraction
    * CNN으로 feature를 변환 : self.conv = nn.Conv2d(2048, hidden_dim, 1)
    * transformer : 인코더와 디코더로 구성
    * 예측 헤드 : class 예측, BBOX 예측 -> 선형 계층
        self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
        self.linear_bbox = nn.Linear(hidden_dim, 4)

* forward
    * 백본으로 ResNet-50을 통해 이미지에서 특징 추출
    * 특징맵 -> CNN -> transformer input : h = self.conv(x) : 2048차원에서 -> 256차원으로 변환
    * positional encoding 구성
    * transformer
    * 출력 예측 (BBOX, class)

### **Q2. Custom Image Detection and Attention Visualization**

In this task, you will upload an **image of your choice** (different from the provided sample) and follow the steps below:

* Object Detection using DETR
 * Use the DETR model to detect objects in your uploaded image.

* Attention Visualization in Encoder
 * Visualize the regions of the image where the encoder focuses the most.

* Decoder Query Attention in Decoder
 * Visualize how the decoder’s query attends to specific areas corresponding to the detected objects.

In [ ]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn


from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()




In this section, we show-case how to load a model from hub, run it on a custom image, and print the result.
Here we load the simplest model (DETR-R50) for fast inference. You can swap it with any other model from the model zoo.

In [ ]:
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model.eval();

#원래 코드 url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
url = 'https://media.istockphoto.com/id/2148372432/ko/%EC%82%AC%EC%A7%84/%EA%B1%B0%EC%8B%A4-%EC%86%8C%ED%8C%8C%EC%97%90%EC%84%9C-%EC%98%81%ED%99%94%EB%A5%BC-%EB%B3%B4%EA%B3%A0-%EC%9E%88%EB%8A%94-%EA%B0%80%EC%A1%B1%EC%9D%98-%EB%92%B7%EB%AA%A8%EC%8A%B5.jpg?s=1024x1024&w=is&k=20&c=JuETFlcC7Z06peL0zEkiF2rDpWLbuIOZDuMF_jAPBH4='
#이미지 출처는 픽사베이(무료이미지)
im = Image.open(requests.get(url, stream=True).raw) # put your own image

# mean-std normalize the input image (batch-size: 1)
img = transform(im).unsqueeze(0)

# propagate through the model
outputs = model(img)

# keep only predictions with 0.7+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
keep = probas.max(-1).values > 0.9

# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

# mean-std normalize the input image (batch-size: 1)
img = transform(im).unsqueeze(0)

# propagate through the model
outputs = model(img)

# keep only predictions with 0.7+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
keep = probas.max(-1).values > 0.9

# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

# mean-std normalize the input image (batch-size: 1)
img = transform(im).unsqueeze(0)

# propagate through the model
outputs = model(img)

# keep only predictions with 0.7+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
keep = probas.max(-1).values > 0.9

# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

plot_results(im, probas[keep], bboxes_scaled)


Here we visualize attention weights of the last decoder layer. This corresponds to visualizing, for each detected objects, which part of the image the model was looking at to predict this specific bounding box and class.

In [ ]:
# use lists to store the outputs via up-values
conv_features, enc_attn_weights, dec_attn_weights = [], [], []

hooks = [
    model.backbone[-2].register_forward_hook(
        lambda self, input, output: conv_features.append(output)
    ),
    model.transformer.encoder.layers[-1].self_attn.register_forward_hook(
        lambda self, input, output: enc_attn_weights.append(output[1])
    ),
    model.transformer.decoder.layers[-1].multihead_attn.register_forward_hook(
        lambda self, input, output: dec_attn_weights.append(output[1])
    ),
]

# propagate through the model
outputs = model(img) # put your own image

for hook in hooks:
    hook.remove() #inference 이후에는 hook 제거. 불필요한 연산 제거

# don't need the list anymore
conv_features = conv_features[0]
enc_attn_weights = enc_attn_weights[0]
dec_attn_weights = dec_attn_weights[0]

In [ ]:
# get the feature map shape
h, w = conv_features['0'].tensors.shape[-2:]

fig, axs = plt.subplots(ncols=len(bboxes_scaled), nrows=2, figsize=(22, 7))
colors = COLORS * 100
for idx, ax_i, (xmin, ymin, xmax, ymax) in zip(keep.nonzero(), axs.T, bboxes_scaled):
    ax = ax_i[0]
    ax.imshow(dec_attn_weights[0, idx].view(h, w))
    ax.axis('off')
    ax.set_title(f'query id: {idx.item()}')
    ax = ax_i[1]
    ax.imshow(im)
    ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                               fill=False, color='blue', linewidth=3))
    ax.axis('off')
    ax.set_title(CLASSES[probas[idx].argmax()])
fig.tight_layout()

**디코더 attention**
1. self attention
    
    * 디코더가 객체 쿼리들 사이의 상호작용을 학습
    * 객체 쿼리들이 서로 독립적으로 작용하지 않음
    * 각 쿼리가 예측하려는 객체가 중복되지 않음

2. encoder - decoder attention

    * 디코더의 객체 쿼리가 인코더가 출력한 특징 맵과 상호작용
    * 디코더의 각 쿼리는 인코더의 특징 맵에서 자신이 집중해야 하는 부분에 대한 정보를 출력
    * 이 과정에서 디코더가 특정 개체에 집중하여 BBOX와 class를 예측

---
---

**디코더 attention 시각화 분석**
* 위의 보라색 이미지

    * 각각의 쿼리 ID의 디코더 attention 가중치를 시각화
    * 밝은 부분(노란색)이 디코더가 해당 객체에 집중하고 있는 부분
    
    * id 25는 person에 집중하고 있는데 위 아래 사진을 비교해보면 위의 사진에서 밝은 부분이 사람

* 아래의 이미지

    * 디코더의 예측결과로, 각 쿼리 ID마다 특정 개체에 집중하여 BBOX와 class를 출력
    * person, potted plant, tv, clock, remote, couch을 분류

    ---

* 특정 객체 쿼리에 대해 이미지 내에서 어떤 영역에 집중하는지를 나타낸다
* 디코더의 self attention으로 각각의 개체가 중복되지 않고 탐지
* 각 쿼리 id에 따라 디코더는 이미지에서 특정 객체에 집중하며 각 객체에 파란색 BBOX와 class를 분류
* 각 id별로 위의 보라색 이미지와 아래의 이미지를 매칭해보면, 각 id의 객체를 탐지하는데 노란색 부분이 가장 큰 도움을 주었다고 해석할 수 있다.
* id 25를 예로 들면 노란색 부분을 보고 'person'이라고 판단했다고 이해 (다른 부분도 봤지만 노란색 부분이 가장 많은 도움이 되었다 정도)

In [ ]:
# output of the CNN
f_map = conv_features['0']
print("Encoder attention:      ", enc_attn_weights[0].shape) # => 인코더의 self attention 가중치! => 얘를 통해 인코더가 입력 이미지에서 어디에 집중하는지 알 수 있따
print("Feature map:            ", f_map.tensors.shape)

In [ ]:
# get the HxW shape of the feature maps of the CNN
shape = f_map.tensors.shape[-2:]
# and reshape the self-attention to a more interpretable shape
sattn = enc_attn_weights[0].reshape(shape + shape)
print("Reshaped self-attention:", sattn.shape)

In [ ]:
# downsampling factor for the CNN, is 32 for DETR and 16 for DETR DC5
fact = 32 #다운샘플링 feature map 크기가 32배 작아짐

# let's select 4 reference points for visualization
idxs = [(200, 200), (280, 400), (200, 600), (440, 800),] # -> 인코더의 self attention이 집중하는 영역을 시각화함

# here we create the canvas
fig = plt.figure(constrained_layout=True, figsize=(25 * 0.7, 8.5 * 0.7))
# and we add one plot per reference point
gs = fig.add_gridspec(2, 4)
axs = [
    fig.add_subplot(gs[0, 0]),
    fig.add_subplot(gs[1, 0]),
    fig.add_subplot(gs[0, -1]),
    fig.add_subplot(gs[1, -1]),
]

# for each one of the reference points, let's plot the self-attention
# for that point
# 인코더 셀프 어텐션 시각화
for idx_o, ax in zip(idxs, axs):
    idx = (idx_o[0] // fact, idx_o[1] // fact)
    ax.imshow(sattn[..., idx[0], idx[1]], cmap='cividis', interpolation='nearest')
    ax.axis('off')
    ax.set_title(f'self-attention{idx_o}')

# and now let's add the central image, with the reference points as red circles
fcenter_ax = fig.add_subplot(gs[:, 1:-1])
fcenter_ax.imshow(im)
for (y, x) in idxs:
    scale = im.height / img.shape[-2]
    x = ((x // fact) + 0.5) * fact
    y = ((y // fact) + 0.5) * fact
    fcenter_ax.add_patch(plt.Circle((x * scale, y * scale), fact // 2, color='r'))
    fcenter_ax.axis('off')

**인코더 attention**
1. self attention

    * 디코더가 객체 쿼리들 사이의 상호작용을 학습
    * 객체 쿼리들이 서로 독립적으로 작용하지 않음
    * 각 쿼리가 예측하려는 객체가 중복되지 않음

---
---

**인코더의 self attention 분석**

* fact = 32 : 실제 이미지보다 32배 작아짐
* 좌우의 파랑노랑 사진들 : 참조포인트에 대한 self attention 가중치를 시각화

    * 노랑부분에 attention이 강하게 작용
    * 인코더가 해당 참조 포인트가 특정 이미지 영역과 관련 있다고 판단하는 부분
* 중간 이미지
    * 빨간색 원 : 참조 포인트
    * 각 참조포인트에 대해 self attention 가중치가 계산됨

    * (200,600)의 경우 원이 tv 근처에 위치해서 왼쪽 위의 self attention(200,600)에서 tv가 있는 곳이 노란색으로 표시됨
* 인코더는 이미지의 전반적인 문맥을 이해
* 각 참조포인트의 노란색 부분은 인코더가 해당 위치와 관련된 특징을 학습했다고 이해
* 즉, (280,400)의 경우 사람의 머리, 주변 환경등 참조포인트의 위치에 대한 특징을 학습함

### **Q3. Understanding Attention Mechanisms**

In this task, you focus on understanding the attention mechanisms present in the encoder and decoder of DETR.

* Briefly describe the types of attention used in the encoder and decoder, and explain the key differences between them.

* Based on the visualized results from Q2, provide an analysis of the distinct characteristics of each attention mechanism in the encoder and decoder. Feel free to express your insights.

1. attention mechanisms of encoder

    * 입력 이미지의 feature map을 처리할 때 self attention을 이용
    * 입력 sequence의 각 요소가 다른 모든 요소와의 관계를 학습하여 전반적인 정보를 파악
    * 즉, 인코더는 이런 매커니즘을 통해 이미지 전체의 특징을 종합적으로 학습할 수 있다.

2. attention mechanisms of decoder

    * self attention

        * 디코더가 객체 쿼리들 사이의 상호작용을 학습
        * 객체 쿼리들이 서로 독립적으로 작용하지 않음
        * 각 쿼리가 예측하려는 객체가 중복되지 않음
    
    * encoder - decoder attention

        * 디코더의 객체 쿼리가 인코더가 출력한 특징 맵과 상호작용
        * 디코더의 각 쿼리는 인코더의 특징 맵에서 자신이 집중해야 하는 부분에 대한 정보를 출력
        * 이 과정에서 디코더가 특정 개체에 집중하여 BBOX와 class를 예측

3. encoder와 decoder의 attention 차이점

    * 인코더는 self attention만 사용하고 디코더는 slef attention과 encoder-decoder attention을 사용
    * 인코더는 self attention만 사용 -> 전체 입력 이미지의 특징을 학습. 이미지의 특징을 요약하고 변환
    * 디코더의 self attention -> 쿼리 사이의 관계를 학습. 서로의 관계를 이해하여 객체의 중복이 없음
    * 디코더의 encoder-decoder attention -> 디코더의 쿼리가 인코더의 특징맵에서 특정 객체와 관련된 정보를 추출
    * 인코더의 목적은 전반적인 특징을 학습하고 요약하여 디코더에 전달하는 것이고
    * 디코더는 인코더에게 받은 특징을 기반으로 object detection하여 class와 BBOX 예측

4. Based on the visualized results from Q2, provide an analysis of the distinct characteristics of each attention mechanism in the encoder and decoder
    
    -> 각 시각화 결과 아래에 분석했습니다
